# DeepAcceptor：Deep learning-based design and screening of non-fullerene acceptor materials for organic solar cells

It is a time-consuming and costly process to develop affordable and high-performance organic photovoltaic materials. Developing reliable computational methods to predict the power conversion efficiency (PCE) is crucial to triage unpromising molecules in large-scale databases and accelerate the material discovery process. In this study, a deep learning-based framework (DeepAcceptor) has been built to design and discover high-efficient small molecule acceptor materials. Specifically, an experimental dataset was constructed by collecting data from publications. Then, a BERT-based model was customized to predict PCEs by taking fully advantages of the atom, bond, connection information in molecular structures of acceptors, and this customized architecture is termed as abcBERT. The computation molecules and experimental molecules were used to pre-train and fine-tune the model, respectively.The molecular graph was used as the input and the computation molecules and experimental molecules were used to pretrain and finetune the model, respectively. DeepAcceptor is a promising method to predict the PCE and speed up the discovery of high-performance acceptor materials.

## Here, we have shown how to use the model to predict the PCE based on NFAs.

### 1. Download the pretrained and finetuned model 

In [19]:
pip install wget

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=bbe0bf5cb9300b74075403b78b87a86314ebb304aff57056f38fc4dcb6bd6fca
  Stored in directory: C:\Users\BM109X32G-10GPU-02\AppData\Local\Temp\pip-ephem-wheel-cache-v97hsxon\wheels\bd\a8\c3\3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


#### download the prediction model

In [20]:
import wget
url = r"https://github.com/JinYSun/DeepAcceptor/releases/download/v1.0.0/data.h5"

In [21]:
wget.download(url,"regression_weights/data.h5")

  3% [##                                                                      ]          664K / 16M

  8% [######                                                                  ]            1M / 16M

 22% [################                                                        ]            3M / 16M

 30% [#####################                                                   ]            5M / 16M

 40% [############################                                            ]            6M / 16M

...

 69% [#################################################                       ]           11M / 16M

 74% [#####################################################                   ]           12M / 16M

 79% [########################################################                ]           13M / 16M

100% [########################################################################]           16M / 16M

'regression_weights/data.h5'

### download the pretrained model

In [22]:
url1 = r"https://github.com/JinYSun/DeepAcceptor/releases/download/v1.0.0/bert_weightsMedium_80.h5"
url2 = r"https://github.com/JinYSun/DeepAcceptor/releases/download/v1.0.0/bert_weights_encoderMedium_80.h5"
wget.download(url1,"medium_weights/bert_weightsMedium_80.h5")
wget.download(url2,"medium_weights/bert_weights_encoderMedium_80.h5")

  4% [###                                                                     ]          792K / 16M

100% [########################################################################]           16M / 16M

'medium_weights/bert_weights_encoderMedium_80.h5'

## Note: Retraining the model is highly recommended to ensure the accuracy of the model!

### 2. Dataset preparation

The atom, bond, connection information were calculated by using rdkit.The training,test and validation dataset are preprocess by runing the utils.py

The preprocess of test dataset is shown as follows.

In [1]:
import utils

import os
from collections import OrderedDict

import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdchem

from compound_constants import DAY_LIGHT_FG_SMARTS_LIST


from utils import mol_to_geognn_graph_data_MMFF3d

In [2]:
    import pandas as pd 
    from tqdm import tqdm
    f = pd.read_csv (r"data/reg/test.csv")
    re = []
    pce = f['PCE']
    for ind, smile in enumerate (tqdm ( f['SMILES'])):
        
        atom,adj = mol_to_geognn_graph_data_MMFF3d(smile)
        print(ind)
        np.save('data/reg/test/adj'+str(ind)+'.npy',np.array(adj))
        re.append([atom,'data/reg/test/adj'+str(ind)+'.npy',pce[ind] ])
    r = pd.DataFrame(re)
    r.to_csv('data/reg/test/test.csv')
    print('Done!')
    

  0%|▍                                                                                 | 1/206 [00:08<28:03,  8.21s/it]

0


  1%|▊                                                                                 | 2/206 [00:09<14:08,  4.16s/it]

1


  1%|█▏                                                                                | 3/206 [00:13<13:25,  3.97s/it]

...

203


100%|█████████████████████████████████████████████████████████████████████████████▌| 205/206 [8:29:46<00:07,  7.36s/it]

204


100%|█████████████████████████████████████████████████████████████████████████████| 206/206 [8:29:53<00:00, 148.51s/it]

205
Done!


### It is recommended to retrain and calculate on the supercomputing!

### 3. Predict

In [3]:
import predict
from predict import *


In [4]:
np.set_printoptions(threshold=sys.maxsize)
prediction_val= main()
print(prediction_val)

data
finish!  Results can be find in abcBERT/results.csv
[[ 6.32487   ]
 [ 5.78203   ]
 [ 9.830467  ]
 [12.525901  ]
 [ 7.6848307 ]
 [ 9.966345  ]
 [ 7.4933352 ]
 [ 9.838319  ]
 [10.137985  ]
 [ 6.458591  ]
 [11.027317  ]
 [ 9.139197  ]
 [10.781679  ]
 [ 5.3456764 ]
 [ 0.9508295 ]
 [ 3.0816107 ]
 [14.772552  ]
 [16.105417  ]
 [10.978567  ]
 [12.361264  ]
 [13.962849  ]
 [ 9.034893  ]
 [16.059265  ]
 [12.31415   ]
 [ 8.812487  ]
 [ 8.208516  ]
 [ 9.085342  ]
 [ 6.809249  ]
 [ 8.577834  ]
 [ 8.696746  ]
 [ 5.345743  ]
 [ 8.35626   ]
 [10.106018  ]
 [13.07176   ]
 [15.83292   ]
 [10.298649  ]
 [ 2.6678524 ]
 [ 8.74655   ]
 [ 8.880405  ]
 [ 7.25626   ]
 [ 1.4840568 ]
 [11.660007  ]
 [11.361352  ]
 [ 8.775978  ]
 [ 6.6474233 ]
 [11.529257  ]
 [ 7.156933  ]
 [ 8.635442  ]
 [ 9.638616  ]
 [11.512436  ]
 [ 9.79162   ]
 [ 8.898342  ]
 [ 3.4168417 ]
 [16.132832  ]
 [10.257622  ]
 [ 8.858661  ]
 [ 5.9179754 ]
 [10.626143  ]
 [14.723402  ]
 [ 9.498344  ]
 [ 8.2845    ]
 [ 6.6272154 ]
 [10.867565  

In [7]:
import predictbysmiles

from predictbysmiles import *

In [9]:
prediction_val = main ('CCCCCCCCC1=CC=C(C2(C3=CC=C(CCCCCCCC)C=C3)C3=CC4=C(C=C3C3=C2C2=C(C=C(C5=CC=C(/C=C6/C(=O)C7=C(C=CC=C7)C6=C(C#N)C#N)C6=NSN=C56)S2)S3)C(C2=CC=C(CCCCCCCC)C=C2)(C2=CC=C(CCCCCCCC)C=C2)C2=C4SC3=C2SC(C2=CC=C(/C=C4\C(=O)C5=C(C=CC=C5)C4=C(C#N)C#N)C4=NSN=C24)=C3)C=C1')

[6.32487]


## Acknowledgement

Jinyu Sun 

E-mail: jinyusun@csu.edu.cn